# 02 - Model Training
Train a credit score prediction model using XGBoost

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load cleaned data
df = pd.read_csv('../data/processed/credit_cleaned.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/credit_cleaned.csv'

In [3]:
import os
os.getcwd()

'C:\\Users\\dacir\\dev\\dr-cre'

In [4]:
df = pd.read_csv('data/processed/credit_cleaned.csv')
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop(columns=['SeriousDlqin2yrs'])
y = df['SeriousDlqin2yrs']

# Split 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train: {X_train.shape[0]} rows')
print(f'Test: {X_test.shape[0]} rows')

In [6]:
## Train XGBoost Model

In [7]:
import xgboost as xgb

# Train model
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)
print('Training complete!')

Training complete!


## Model Evaluation

In [8]:
from sklearn.metrics import classification_report

# Predict
y_pred = model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     20936
           1       0.54      0.12      0.20      1377

    accuracy                           0.94     22313
   macro avg       0.74      0.56      0.58     22313
weighted avg       0.92      0.94      0.92     22313



## Evaluation Results

- **Accuracy 94%**: Looks good but misleading. Model can get 94% by always predicting "no default"
- **Precision (class 1) 54%**: When model predicts default, it's correct 54% of the time
- **Recall (class 1) 12%**: Model only catches 12% of actual defaulters - too low
- **F1-score (class 1) 0.20**: Poor balance between precision and recall
- **Root cause**: Dataset is imbalanced - 83,979 non-default vs 5,272 default (16x difference)
- **Solution**: Use scale_pos_weight to give more weight to minority class

In [9]:
# Check class ratio
print(y_train.value_counts())

SeriousDlqin2yrs
0    83979
1     5272
Name: count, dtype: int64


In [10]:
# 83979 / 5272 = 16 (non-default is 16x more than default)
ratio = 83979 / 5272

# Give more weight to minority class (defaulted)
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=ratio  # Balance the imbalanced dataset
)

model.fit(X_train, y_train)
print('Training complete!')

Training complete!


In [11]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88     20936
           1       0.19      0.73      0.30      1377

    accuracy                           0.79     22313
   macro avg       0.58      0.77      0.59     22313
weighted avg       0.93      0.79      0.84     22313



## After Applying scale_pos_weight

| | Before | After |
|---|---|---|
| Recall (class 1) | 12% | 73% |
| Precision (class 1) | 54% | 19% |
| Accuracy | 94% | 79% |

**Why?** Model now aggressively predicts default to catch more real defaulters.
**Trade-off:** High recall = catches more defaulters, but more false alarms.
**Our choice:** Recall matters more for a credit coach app.

## Save Model

In [12]:
import joblib

# Save trained model
joblib.dump(model, 'models/credit_model.pkl')
print('Model saved!')

Model saved!


## Credit Score Conversion
Convert default probability (0 to 1) to FICO-style score (300 to 850)

In [13]:
import numpy as np

def probability_to_score(probability):
    # Higher probability of default = lower score
    # Lower probability of default = higher score
    score = 850 - (probability * (850 - 300))
    return int(score)

# Test
print(probability_to_score(0.0))   # Best case
print(probability_to_score(0.5))   # Average
print(probability_to_score(1.0))   # Worst case

850
575
300


In [14]:
# Get default probability for test data
probabilities = model.predict_proba(X_test)[:, 1]

# Convert to credit scores
scores = [probability_to_score(p) for p in probabilities]

# Preview
print(f'Min score: {min(scores)}')
print(f'Max score: {max(scores)}')
print(f'Average score: {int(sum(scores)/len(scores))}')

Min score: 309
Max score: 842
Average score: 670


## Score Distribution
- Min: 309 | Max: 842 | Average: 670
- US average FICO score is around 670~680 (matches real data!)